In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240706'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/7.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Type No'] = input_['Type No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            import requests

            import sys
            sys.path.append('../00.Tools')
            from crawler_configuration import get_header, get_proxy

            from bs4 import BeautifulSoup
            from lxml import etree

            list_url = []
            while len(list_url) < input_.loc[a, 'SKU Count']:
                for i in range(input_.loc[a, 'Page']):
                    for j in range(30):
                        try:
                            resp = requests.get(f'''{input_.loc[a, 'Type Url']}&start={i*100}&num=100''',
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10)).text

                            # = = = = = = = = = = = = = = =

                            soup = BeautifulSoup(resp.strip(), 'lxml')
                            html = etree.HTML(str(soup))

                            # = = = = = = = = = = = = = = =

                            list_url += [f'https://www.dormanproducts.com/{href.strip()}' for href in html.xpath('//h2[@class="item-headline"]/a/@href')]
                            list_url = sorted(list(set(list_url)), key=list_url.index)

                            break
                        except:
                            continue

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Type No': input_.loc[a, 'Type No'],
                                    'Type': input_.loc[a, 'Type'],
                                    'Type Url': input_.loc[a, 'Type Url'],
                                    'SKU Count': input_.loc[a, 'SKU Count'],
                                    'Page': input_.loc[a, 'Page'],
                                    'No': [j+1 for j in range(len(list_url))],
                                    'Url': list_url})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'

        # = = = = = = = = = = = = = = =

        except:
            pass

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type Url': input_.loc[a, 'Type Url'],
                                     'SKU Count': input_.loc[a, 'SKU Count'],
                                     'Page': input_.loc[a, 'Page']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}] - {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/8.url_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Type No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/url_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok] - 1272.Luggage Compartment Light Bulb
[剩余数量：1000] - [当前时间：18:47:51]

[状态：ok] - 1273.Manifold Absolute Pressure (MAP) Sensor
[剩余数量：999] - [当前时间：18:47:59]

[状态：ok] - 1048.Handle - Exterior Door
[剩余数量：998] - [当前时间：18:48:02]

[状态：ok] - 1275.Manual Transmission Output Shaft Speed Sensor
[剩余数量：997] - [当前时间：18:48:06]

[状态：ok] - 1274.Manual Transmission Drain Plug Gasket
[剩余数量：996] - [当前时间：18:48:09]

[状态：ok] - 1276.Manual Transmission Shift Cable Bushing
[剩余数量：995] - [当前时间：18:48:09]

[状态：ok] - 1258.Liftgate Latch
[剩余数量：994] - [当前时间：18:48:11]

[状态：ok] - 1279.Manual Transmission Shift Tower Kit
[剩余数量：993] - [当前时间：18:48:14]

[状态：ok] - 1277.Manual Transmission Shift Knob
[剩余数量：992] - [当前时间：18:48:14]

[状态：ok] - 1278.Manual Transmission Shift Lever Bushing
[剩余数量：991] - [当前时间：18:48:14]

[状态：ok] - 1281.Manual Transmission Shifter Repair Kit
[剩余数量：990] - [当前时间：18:48:17]

[状态：ok] - 1280.Manual Transmission Shifter Linkage
[剩余数量：989] - [当前时间：18:48:19]

[状态：ok] - 1283.Map Light Wiring Harness
[剩余数